In [35]:
import pandas as pd

In [36]:
df = pd.read_csv(r"C:\Users\qls05\OneDrive\바탕 화면\df.csv", encoding = 'cp949')

df['preprocessed_송출내용'] = df['preprocessed_송출내용'].fillna('')
texts = df['preprocessed_송출내용']

### 주어진 df ->  tf-idf 벡터화

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=30)
vectored_df = vectorizer.fit_transform(texts)

In [38]:
dense_df = vectored_df.todense() #vectored_df는 희소행렬이기 때문에 dense 형태로 전환.

feature_names = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(dense_df, columns=feature_names)

### 데이터 분할

In [39]:
from sklearn.model_selection import train_test_split

X = df_tfidf
y = df['label']

# train : val : test = 6 : 2 : 2
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) 


print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape, "y_valid shape:", y_valid.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (5608, 30) y_train shape: (5608,)
X_valid shape: (1870, 30) y_valid shape: (1870,)
X_test shape: (1870, 30) y_test shape: (1870,)


# ----------------------------------------------------------------------------

## CatBoost모델 적용

CatBoost는 Yandex에서 개발한 머신 러닝 라이브러리로, 주로 분류와 회귀 작업에 사용되는 그라디언트 부스팅 결정 트리(GBDT) 알고리즘입니다. "CatBoost"는 "Category"와 "Boosting"을 결합한 용어로, 특히 카테고리형 데이터를 효과적으로 처리할 수 있는 능력이 특징입니다. CatBoost는 다른 인기 있는 그라디언트 부스팅 라이브러리인 XGBoost와 LightGBM과 비교할 때 몇 가지 독특한 장점을 가지고 있습니다.

CatBoost의 주요 특징
카테고리 특성 처리: CatBoost는 카테고리형 특성을 자동으로 변환하여 수치형 데이터로 인코딩합니다. 이는 별도의 전처리 과정 없이도 모델 입력으로 직접 사용할 수 있게 해줍니다. 이 기능은 카테고리형 데이터가 많은 실세계 데이터셋에 매우 유용합니다.

Order in Boosting (순서 민감 부스팅): CatBoost는 훈련 데이터의 순서를 변경하여 여러 번 모델을 훈련시킴으로써 과적합을 방지합니다. 이는 "Ordered Boosting"이라는 기술로, 데이터 순서에 따른 변동성을 감소시켜 모델의 일반화 성능을 향상시킵니다.

속도와 확장성: CatBoost는 GPU와 멀티 CPU 환경에서 높은 성능을 발휘하며, 대용량 데이터셋 처리에 효율적입니다. 이는 병렬 처리와 캐시 최적화를 통해 달성됩니다.

직관적인 하이퍼파라미터: CatBoost의 하이퍼파라미터는 상대적으로 이해하기 쉽고, 기본 설정만으로도 높은 성능을 내는 경우가 많습니다. 이로 인해 머신 러닝 초보자도 쉽게 접근할 수 있습니다.

Python, R, Java 등 다양한 언어 지원: CatBoost는 Python 뿐만 아니라 R, Java 등 여러 프로그래밍 언어를 지원하여 다양한 환경에서 사용할 수 있습니다.

In [40]:
pip install catboost

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [42]:
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report

catboost = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClass',  # 다중 클래스 로스 함수 설정
    verbose=True,
    early_stopping_rounds=50
)
catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid))


y_valid_hat = catboost.predict(X_valid)
valid_accuracy = accuracy_score(y_valid, y_valid_hat)

print("valid score: %.3f" %valid_accuracy)

0:	learn: 1.7987805	test: 1.8165095	best: 1.8165095 (0)	total: 116ms	remaining: 1m 55s
1:	learn: 1.6169069	test: 1.6426960	best: 1.6426960 (1)	total: 193ms	remaining: 1m 36s
2:	learn: 1.4905398	test: 1.5228083	best: 1.5228083 (2)	total: 277ms	remaining: 1m 32s
3:	learn: 1.3933041	test: 1.4298641	best: 1.4298641 (3)	total: 357ms	remaining: 1m 28s
4:	learn: 1.3063546	test: 1.3480726	best: 1.3480726 (4)	total: 437ms	remaining: 1m 26s
5:	learn: 1.2351357	test: 1.2816984	best: 1.2816984 (5)	total: 511ms	remaining: 1m 24s
6:	learn: 1.1780929	test: 1.2292144	best: 1.2292144 (6)	total: 581ms	remaining: 1m 22s
7:	learn: 1.1281418	test: 1.1826242	best: 1.1826242 (7)	total: 675ms	remaining: 1m 23s
8:	learn: 1.0825362	test: 1.1388175	best: 1.1388175 (8)	total: 753ms	remaining: 1m 22s
9:	learn: 1.0433000	test: 1.1017772	best: 1.1017772 (9)	total: 839ms	remaining: 1m 23s
10:	learn: 1.0093940	test: 1.0699520	best: 1.0699520 (10)	total: 935ms	remaining: 1m 24s
11:	learn: 0.9772874	test: 1.0412914	best

93:	learn: 0.5345678	test: 0.6452938	best: 0.6452938 (93)	total: 8.07s	remaining: 1m 17s
94:	learn: 0.5337829	test: 0.6446305	best: 0.6446305 (94)	total: 8.15s	remaining: 1m 17s
95:	learn: 0.5330317	test: 0.6440765	best: 0.6440765 (95)	total: 8.22s	remaining: 1m 17s
96:	learn: 0.5320439	test: 0.6432336	best: 0.6432336 (96)	total: 8.31s	remaining: 1m 17s
97:	learn: 0.5305230	test: 0.6422707	best: 0.6422707 (97)	total: 8.38s	remaining: 1m 17s
98:	learn: 0.5293532	test: 0.6417104	best: 0.6417104 (98)	total: 8.45s	remaining: 1m 16s
99:	learn: 0.5286301	test: 0.6413879	best: 0.6413879 (99)	total: 8.53s	remaining: 1m 16s
100:	learn: 0.5274912	test: 0.6407346	best: 0.6407346 (100)	total: 8.61s	remaining: 1m 16s
101:	learn: 0.5264388	test: 0.6398703	best: 0.6398703 (101)	total: 8.69s	remaining: 1m 16s
102:	learn: 0.5260335	test: 0.6395622	best: 0.6395622 (102)	total: 8.78s	remaining: 1m 16s
103:	learn: 0.5250492	test: 0.6390773	best: 0.6390773 (103)	total: 8.86s	remaining: 1m 16s
104:	learn: 0

184:	learn: 0.4717710	test: 0.6074540	best: 0.6074540 (184)	total: 16s	remaining: 1m 10s
185:	learn: 0.4707926	test: 0.6065640	best: 0.6065640 (185)	total: 16.1s	remaining: 1m 10s
186:	learn: 0.4700872	test: 0.6061684	best: 0.6061684 (186)	total: 16.2s	remaining: 1m 10s
187:	learn: 0.4695492	test: 0.6059687	best: 0.6059687 (187)	total: 16.2s	remaining: 1m 10s
188:	learn: 0.4686528	test: 0.6050794	best: 0.6050794 (188)	total: 16.3s	remaining: 1m 10s
189:	learn: 0.4685216	test: 0.6049720	best: 0.6049720 (189)	total: 16.4s	remaining: 1m 9s
190:	learn: 0.4683308	test: 0.6049094	best: 0.6049094 (190)	total: 16.5s	remaining: 1m 9s
191:	learn: 0.4677772	test: 0.6047757	best: 0.6047757 (191)	total: 16.6s	remaining: 1m 9s
192:	learn: 0.4670082	test: 0.6041039	best: 0.6041039 (192)	total: 16.7s	remaining: 1m 9s
193:	learn: 0.4668045	test: 0.6041789	best: 0.6041039 (192)	total: 16.8s	remaining: 1m 9s
194:	learn: 0.4661883	test: 0.6040268	best: 0.6040268 (194)	total: 16.9s	remaining: 1m 9s
195:	le

277:	learn: 0.4344340	test: 0.5883728	best: 0.5883728 (277)	total: 24.5s	remaining: 1m 3s
278:	learn: 0.4342364	test: 0.5883910	best: 0.5883728 (277)	total: 24.6s	remaining: 1m 3s
279:	learn: 0.4336794	test: 0.5878863	best: 0.5878863 (279)	total: 24.7s	remaining: 1m 3s
280:	learn: 0.4333744	test: 0.5878519	best: 0.5878519 (280)	total: 24.7s	remaining: 1m 3s
281:	learn: 0.4331230	test: 0.5877611	best: 0.5877611 (281)	total: 24.8s	remaining: 1m 3s
282:	learn: 0.4328834	test: 0.5877386	best: 0.5877386 (282)	total: 24.9s	remaining: 1m 3s
283:	learn: 0.4327644	test: 0.5877948	best: 0.5877386 (282)	total: 25s	remaining: 1m 2s
284:	learn: 0.4325725	test: 0.5877050	best: 0.5877050 (284)	total: 25s	remaining: 1m 2s
285:	learn: 0.4325033	test: 0.5877181	best: 0.5877050 (284)	total: 25.1s	remaining: 1m 2s
286:	learn: 0.4323358	test: 0.5874645	best: 0.5874645 (286)	total: 25.2s	remaining: 1m 2s
287:	learn: 0.4322771	test: 0.5874299	best: 0.5874299 (287)	total: 25.3s	remaining: 1m 2s
288:	learn: 0.

370:	learn: 0.4103107	test: 0.5787927	best: 0.5786140 (366)	total: 32.6s	remaining: 55.3s
371:	learn: 0.4102792	test: 0.5787714	best: 0.5786140 (366)	total: 32.7s	remaining: 55.1s
372:	learn: 0.4098078	test: 0.5784832	best: 0.5784832 (372)	total: 32.8s	remaining: 55.1s
373:	learn: 0.4095979	test: 0.5784034	best: 0.5784034 (373)	total: 32.8s	remaining: 54.9s
374:	learn: 0.4094089	test: 0.5783424	best: 0.5783424 (374)	total: 32.9s	remaining: 54.8s
375:	learn: 0.4093121	test: 0.5783174	best: 0.5783174 (375)	total: 33s	remaining: 54.7s
376:	learn: 0.4091697	test: 0.5783801	best: 0.5783174 (375)	total: 33.1s	remaining: 54.6s
377:	learn: 0.4089062	test: 0.5784830	best: 0.5783174 (375)	total: 33.1s	remaining: 54.5s
378:	learn: 0.4087736	test: 0.5783701	best: 0.5783174 (375)	total: 33.2s	remaining: 54.4s
379:	learn: 0.4084760	test: 0.5782329	best: 0.5782329 (379)	total: 33.3s	remaining: 54.3s
380:	learn: 0.4084022	test: 0.5782111	best: 0.5782111 (380)	total: 33.4s	remaining: 54.2s
381:	learn: 

...

557:	learn: 0.3756666	test: 0.5700693	best: 0.5697900 (544)	total: 49.6s	remaining: 39.3s
558:	learn: 0.3755267	test: 0.5698473	best: 0.5697900 (544)	total: 49.7s	remaining: 39.2s
559:	learn: 0.3754799	test: 0.5697798	best: 0.5697798 (559)	total: 49.8s	remaining: 39.1s
560:	learn: 0.3752021	test: 0.5697053	best: 0.5697053 (560)	total: 49.9s	remaining: 39s
561:	learn: 0.3751260	test: 0.5698092	best: 0.5697053 (560)	total: 50s	remaining: 38.9s
562:	learn: 0.3751004	test: 0.5697856	best: 0.5697053 (560)	total: 50.1s	remaining: 38.9s
563:	learn: 0.3749676	test: 0.5696502	best: 0.5696502 (563)	total: 50.2s	remaining: 38.8s
564:	learn: 0.3749186	test: 0.5696567	best: 0.5696502 (563)	total: 50.3s	remaining: 38.7s
565:	learn: 0.3747221	test: 0.5694464	best: 0.5694464 (565)	total: 50.4s	remaining: 38.6s
566:	learn: 0.3745102	test: 0.5693448	best: 0.5693448 (566)	total: 50.4s	remaining: 38.5s
567:	learn: 0.3743318	test: 0.5692785	best: 0.5692785 (567)	total: 50.5s	remaining: 38.4s
568:	learn: 0.

649:	learn: 0.3625255	test: 0.5671601	best: 0.5671057 (642)	total: 58.4s	remaining: 31.5s
650:	learn: 0.3623475	test: 0.5672827	best: 0.5671057 (642)	total: 58.5s	remaining: 31.4s
651:	learn: 0.3622329	test: 0.5672498	best: 0.5671057 (642)	total: 58.6s	remaining: 31.3s
652:	learn: 0.3620534	test: 0.5673041	best: 0.5671057 (642)	total: 58.7s	remaining: 31.2s
653:	learn: 0.3619911	test: 0.5671972	best: 0.5671057 (642)	total: 58.8s	remaining: 31.1s
654:	learn: 0.3619146	test: 0.5671319	best: 0.5671057 (642)	total: 58.8s	remaining: 31s
655:	learn: 0.3617836	test: 0.5669575	best: 0.5669575 (655)	total: 58.9s	remaining: 30.9s
656:	learn: 0.3617015	test: 0.5669949	best: 0.5669575 (655)	total: 59s	remaining: 30.8s
657:	learn: 0.3616841	test: 0.5670076	best: 0.5669575 (655)	total: 59.1s	remaining: 30.7s
658:	learn: 0.3614811	test: 0.5666735	best: 0.5666735 (658)	total: 59.3s	remaining: 30.7s
659:	learn: 0.3612828	test: 0.5666565	best: 0.5666565 (659)	total: 59.4s	remaining: 30.6s
660:	learn: 0.

741:	learn: 0.3515017	test: 0.5647318	best: 0.5647008 (719)	total: 1m 8s	remaining: 23.8s
742:	learn: 0.3514395	test: 0.5648127	best: 0.5647008 (719)	total: 1m 8s	remaining: 23.7s
743:	learn: 0.3514045	test: 0.5648670	best: 0.5647008 (719)	total: 1m 8s	remaining: 23.6s
744:	learn: 0.3512019	test: 0.5648897	best: 0.5647008 (719)	total: 1m 8s	remaining: 23.6s
745:	learn: 0.3511171	test: 0.5648681	best: 0.5647008 (719)	total: 1m 9s	remaining: 23.6s
746:	learn: 0.3510919	test: 0.5648712	best: 0.5647008 (719)	total: 1m 9s	remaining: 23.5s
747:	learn: 0.3509764	test: 0.5648152	best: 0.5647008 (719)	total: 1m 9s	remaining: 23.4s
748:	learn: 0.3509149	test: 0.5648469	best: 0.5647008 (719)	total: 1m 9s	remaining: 23.4s
749:	learn: 0.3508739	test: 0.5648692	best: 0.5647008 (719)	total: 1m 10s	remaining: 23.4s
750:	learn: 0.3506772	test: 0.5647223	best: 0.5647008 (719)	total: 1m 10s	remaining: 23.3s
751:	learn: 0.3506315	test: 0.5647233	best: 0.5647008 (719)	total: 1m 10s	remaining: 23.3s
752:	le

833:	learn: 0.3413733	test: 0.5633223	best: 0.5632848 (832)	total: 1m 44s	remaining: 20.7s
834:	learn: 0.3411591	test: 0.5633767	best: 0.5632848 (832)	total: 1m 44s	remaining: 20.6s
835:	learn: 0.3411013	test: 0.5633318	best: 0.5632848 (832)	total: 1m 44s	remaining: 20.5s
836:	learn: 0.3410095	test: 0.5634233	best: 0.5632848 (832)	total: 1m 44s	remaining: 20.3s
837:	learn: 0.3409774	test: 0.5634240	best: 0.5632848 (832)	total: 1m 44s	remaining: 20.2s
838:	learn: 0.3408990	test: 0.5633717	best: 0.5632848 (832)	total: 1m 44s	remaining: 20.1s
839:	learn: 0.3407996	test: 0.5633058	best: 0.5632848 (832)	total: 1m 44s	remaining: 19.9s
840:	learn: 0.3406725	test: 0.5631972	best: 0.5631972 (840)	total: 1m 44s	remaining: 19.8s
841:	learn: 0.3406453	test: 0.5632000	best: 0.5631972 (840)	total: 1m 44s	remaining: 19.7s
842:	learn: 0.3404826	test: 0.5631112	best: 0.5631112 (842)	total: 1m 44s	remaining: 19.5s
843:	learn: 0.3403866	test: 0.5631483	best: 0.5631112 (842)	total: 1m 45s	remaining: 19.4s

924:	learn: 0.3328145	test: 0.5622487	best: 0.5622349 (923)	total: 1m 53s	remaining: 9.22s
925:	learn: 0.3327702	test: 0.5622244	best: 0.5622244 (925)	total: 1m 53s	remaining: 9.1s
926:	learn: 0.3327113	test: 0.5622215	best: 0.5622215 (926)	total: 1m 53s	remaining: 8.97s
927:	learn: 0.3326408	test: 0.5622678	best: 0.5622215 (926)	total: 1m 54s	remaining: 8.85s
928:	learn: 0.3325677	test: 0.5622651	best: 0.5622215 (926)	total: 1m 54s	remaining: 8.72s
929:	learn: 0.3325432	test: 0.5623071	best: 0.5622215 (926)	total: 1m 54s	remaining: 8.6s
930:	learn: 0.3322999	test: 0.5622301	best: 0.5622215 (926)	total: 1m 54s	remaining: 8.47s
931:	learn: 0.3322690	test: 0.5622115	best: 0.5622115 (931)	total: 1m 54s	remaining: 8.35s
932:	learn: 0.3321555	test: 0.5622641	best: 0.5622115 (931)	total: 1m 54s	remaining: 8.22s
933:	learn: 0.3321389	test: 0.5622756	best: 0.5622115 (931)	total: 1m 54s	remaining: 8.1s
934:	learn: 0.3321075	test: 0.5623055	best: 0.5622115 (931)	total: 1m 54s	remaining: 7.97s
93

In [43]:
print("valid score: %.3f" %valid_accuracy)

valid score: 0.822


In [44]:
#상세 보고서
print(metrics.classification_report(y_valid, y_valid_hat))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       152
           1       0.69      0.72      0.70       162
           2       0.63      0.63      0.63        89
           3       0.62      0.49      0.55        51
           4       0.81      0.81      0.81       607
           5       0.93      0.92      0.93       464
           6       0.78      0.87      0.82       194
           7       0.81      0.75      0.78       151

    accuracy                           0.82      1870
   macro avg       0.78      0.76      0.77      1870
weighted avg       0.82      0.82      0.82      1870



## hyperparmeter 튜닝

**1. grid search로 탐색 >> 너~~무 느림**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'iterations': [500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 10],
    'l2_leaf_reg': [1, 3, 5]
}
catboost = CatBoostClassifier(early_stopping_rounds=50, verbose=False)
grid_search = GridSearchCV(catboost, param_grid, cv=2, scoring='accuracy')

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score: {:.2f}".format(grid_search.best_score_))

**2. for문으로도 해봄**

In [50]:
learning_rate = [0.01, 0.05, 0.1]
depth = [4, 6, 10]
l2_leaf_reg = [1, 3, 5]
results = []

for lr in learning_rate:
    for d in depth:
        for leaf in l2_leaf_reg:
            catboost = CatBoostClassifier(learning_rate=lr, depth=d,l2_leaf_reg=leaf,
                                          loss_function='MultiClass',iterations=500, early_stopping_rounds=50, verbose=False).fit(X_train, y_train)
    
            y_train_hat = catboost.predict(X_train)
            y_valid_hat =catboost.predict(X_valid)
        
            train_accuracy = accuracy_score(y_train, y_train_hat)
            valid_accuracy = accuracy_score(y_valid, y_valid_hat)


            results.append({'learning_rate': lr,
                            'depth': d,
                            'l2_leaf_reg': leaf,
                            'valid_accuracy': valid_accuracy})


display(pd.DataFrame(results))

KeyboardInterrupt: 

#### 다 너무 느려서 튜닝 안함 일단 코드만 올림ㅡㅡ

## test score 도출

In [29]:
catboost('''여기에 hyperparameter 입력''').fit(X_train, y_train, eval_set=(X_test, y_test))

y_test_hat =svc.predict(X_test)
        
test_accuracy = accuracy_score(y_test, y_test_hat)
print("test score: %.3f" %test_accuracy)

test score: 0.839


In [ ]:
#상세 보고서
print(metrics.classification_report(y_test, y_test_hat))